# Exploration des données de Sirene

## Importation des différents packages nécessaires.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import nltk
import string
nltk.download('stopwords')

## Importation des données

In [ ]:
DBRaw = pd.read_parquet('../data/extraction_sirene_20220510.parquet', engine='pyarrow')

On transforme les valeurs manquantes en NaN.

In [ ]:
DBRaw = DBRaw.fillna(value=np.nan)

On garde seulement les variables potentiellement intéressantes : 
- ``APE_SICORE :`` Code APE (Activité Principale Exercée) retenu lors du traitement de codification (soit Sicore soit gestionnaire) ;
- ``NAT_SICORE :`` Nature de l'activité de l'entreprise ;
- ``SED_SICORE :`` Sédentarité de l'entreprise ;
- ``EVT_SICORE :`` Sédentarité de l'entreprise ;
- ``LIB_SICORE :`` Sédentarité de l'entreprise ;
- ``DATE :`` Sédentarité de l'entreprise ;
- ``AUTO :`` Type de liasse extrait de la base brute Sirène ;
- ``SURF :`` Surface en $m^2$ de l'établissement.


In [ ]:
Var2Keep = ["APE_SICORE","LIB_SICORE","AUTO","DATE","NAT_SICORE","SED_SICORE","EVT_SICORE","SURF"]
DB = DBRaw[Var2Keep]

On regarde l'ampleur des variables manquantes pour les deux variables principales ``APE_SICORE`` et ``LIB_SICORE``.

In [ ]:
DB[["APE_SICORE","LIB_SICORE"]].isnull().sum()/DB.shape[0] * 100

On supprime les liasses où une valeur est manquante pour l'une de ces deux variables (6.77% de la base). Il s'agit principalement du code APE donc il n'est pas nécessaire de l'imputer.

In [ ]:
DB = DB[DB["APE_SICORE"].notna()]

On a finalement 10.8 millions de liasses.

In [ ]:
DB.shape

## I- Modèle 1

On estime un modèle FastText standard en utilisant seulement les libellés comme features.

In [ ]:
DB1["LIB_SICORE"].isnull().sum()

In [ ]:
DB1 = DB[["APE_SICORE","LIB_SICORE"]]

In [ ]:
sstr = "Contruction, EN l'INNOVATION au _78"

In [ ]:
sstr = sstr.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
sstr = sstr.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

In [ ]:
def CleanLib(lib):
    if not isinstance(lib,str):
        return "NAN"
    else:
        # On supprime toutes les ponctuations
        lib = lib.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
        # On supprime tous les chiffres
        lib = lib.translate(str.maketrans(string.digits, ' ' * len(string.digits)))

        # On supprime les stopwords et on renvoie les mots en majuscule
        return " ".join([x.upper() for x in lib.split() if x.lower() not in stopwords.words('french') + ['a']])


In [ ]:
from tqdm import tqdm

In [ ]:
import time

In [ ]:
temp = DB1.head(100)

In [ ]:
t0 = time.time()
temp["LIB_SICORE"].apply(lambda x: CleanLib(x))
t1 = time.time()

t1-t0

In [ ]:
t0 = time.time()
[CleanLib(x) for x in tqdm(DB1["LIB_SICORE"].tolist())]
t1 = time.time()

t1-t0